# TERRA Colocolizes to Double Stranded Breaks (DSBs), esp. in G2
    This project is the data analysis of Dr. Taghreed Mohammed Al Turki's PhD data, in continuation of Chris Nelson's(?) previous work. The objective is to eelucidate if TERRA locolizes to global DSBs, to telomeric DSB's, and which phase of the cell cycle does TERRA colocolize to telo DSBs. 


In [1]:
# handling data
import pandas as pd
import numpy as np

# visualizing data
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

# Statistics
from scipy import stats
import statsmodels.api as sm
import statsmodels.imputation.mice as smi
import scipy.cluster.hierarchy as hac
import matplotlib.gridspec as gridspec
import math
import random

#SHAP Value Stuff
import shap
from shap import TreeExplainer
from shap import GradientExplainer
from shap import DeepExplainer
from shap import KernelExplainer
from shap import SamplingExplainer
from shap import PartitionExplainer

print("Setup Complete")

Setup Complete


In [2]:
pwd

'/Users/aidanlew'

In [2]:
cd '/Users/aidanlew/Documents'

/Users/aidanlew/Documents


In [5]:
pwd

'/Users/aidanlew/Desktop/senior_end/tagh_stuff'

# Setting up the data
    We want the data to be read by the program environment so that we can run quick and accurate analysis further on.

In [26]:
### REMEBER TO SAVE AS CSV FOR GITHUB ###

#Data for Global DSB analyis
global_df = pd.read_excel('../Documents/global_DSB_TERRA.xlsx',
                    sheet_name=0,
                    skiprows = 0,
                    usecols = ['foci#_TERRA_with_2Gy ', 'TERRA_foci#/cell_with_0Gy_(control)',
       'foci#_TERRA_2Gy_+RnaseA+H', 'foci#_TERRA_0Gy_+RnaseA+H'] #Mind the space in first column name
                         )

global_df.set_index(['foci#_TERRA_with_2Gy '])#good way to fix "not in axis issue"
global_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#global_df.columns() #Fr some reason not setting the index to the columnn name
global_df.head()



,foci#_TERRA_with_2Gy,TERRA_foci#/cell_with_0Gy_(control),foci#_TERRA_2Gy_+RnaseA+H,foci#_TERRA_0Gy_+RnaseA+H
0,14,2,1.0,8
1,24,2,0.0,5
2,19,1,3.0,5
3,43,0,1.0,8
4,17,11,1.0,6


In [ ]:
#Data for telo DSB analyis
telo_df = pd.read_excel('../Documents/global_DSB_TERRA.xlsx',
                    sheet_name=0,
                    skiprows = 0,
                    usecols = ['foci#_TERRA_with_2Gy ', 'TERRA_foci#/cell_with_0Gy_(control)',
       'foci#_TERRA_2Gy_+RnaseA+H', 'foci#_TERRA_0Gy_+RnaseA+H'] 
                         )

telo_df.set_index(['foci#_TERRA_with_2Gy '])#good way to fix "not in axis issue"
telo_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#telo_df.columns()
telo_df.head()

In [31]:
#Data for G2 DSB analyis

#for total TERRA loci number
g2_terra_num_df = pd.read_excel('../Documents/telomere_G2_DSB_TERRA.xlsx',
                    sheet_name=0,
                    skiprows = 0,
                    usecols = ['Cells', 'no_RNAse_G1',
       'no_RNAse_S', 'no_RNAse_G2','no_RNAse_total','RNAse_G1','RNAse_S','RNAse_G2','RNAse_total'] 
                         )

g2_terra_num_df.set_index(['Cells'])#good way to fix "not in axis issue"
g2_terra_num_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#g2_terra_num_df.columns()
g2_terra_num_df.head()

,Cells,no_RNAse_G1,no_RNAse_S,no_RNAse_G2,no_RNAse_total,RNAse_G1,RNAse_S,RNAse_G2,RNAse_total
0,EN-T,16.1,12.8,55.0,83.9,19.6,13.6,53.9,87.1
1,TRF1,14.3,8.1,63.5,85.9,22.7,13.3,48.1,84.1
2,Empty,22.6,6.8,57.3,86.7,24.7,13.5,30.8,69.0
3,Untransfected,40.4,13.2,50.9,104.5,59.2,21.7,29.7,110.6


In [32]:
# For total cell loci number
g2_cell_num_df = pd.read_excel('../Documents/telomere_G2_DSB_TERRA.xlsx',
                    sheet_name=2,
                    skiprows = 0,
                    usecols = ['Cells', 'no_RNAse_G1',
       'no_RNAse_S', 'no_RNAse_G2','no_RNAse_total','RNAse_G1','RNAse_S','RNAse_G2','RNAse_total'] 
                         )

g2_cell_num_df.set_index(['Cells'])#good way to fix "not in axis issue"
g2_cell_num_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#g2_cell_num_df.columns()
g2_cell_num_df.head()

,Cells,no_RNAse_G1,no_RNAse_S,no_RNAse_G2,no_RNAse_total,RNAse_G1,RNAse_S,RNAse_G2,RNAse_total
0,EN-T,16.1,12.8,55.0,83.9,19.6,13.6,53.9,87.1
1,TRF1,14.3,8.1,63.5,85.9,22.7,13.3,48.1,84.1
2,Empty,22.6,6.8,57.3,86.7,24.7,13.5,30.8,69.0
3,Untransfected,40.4,13.2,50.9,104.5,59.2,21.7,29.7,110.6


In [33]:
# For percent of total cells in the cell cyle phase
g2_cell_perc_df = pd.read_excel('../Documents/telomere_G2_DSB_TERRA.xlsx',
                    sheet_name=3,
                    skiprows = 0,
                    usecols = ['Cells', 'no_RNAse_G1',
       'no_RNAse_S', 'no_RNAse_G2','RNAse_G1','RNAse_S','RNAse_G2'] 
                         )

g2_cell_perc_df.set_index(['Cells'])#good way to fix "not in axis issue"
g2_cell_perc_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#g2_cell_perc_df.columns()
g2_cell_perc_df.head()

,Cells,no_RNAse_G1,no_RNAse_S,no_RNAse_G2,RNAse_G1,RNAse_S,RNAse_G2
0,EN-T,19.189511,15.256257,65.554231,22.502870,15.614237,61.882893
1,TRF1,16.647264,9.429569,73.923166,26.991677,15.814507,57.193817
2,Empty,26.066897,7.843137,66.089965,35.797101,19.565217,44.637681
3,Untransfected,38.660287,12.631579,48.708134,53.526221,19.620253,26.853526


In [35]:
#For the percent of TERRA loci in each phase of the cell cycle
# For percent of total cells in the cell cyle phase
g2_terra_perc_df = pd.read_excel('../Documents/telomere_G2_DSB_TERRA.xlsx',
                    sheet_name=1,
                    skiprows = 0,
                    usecols = ['Cells', 'no_RNAse_G1',
       'no_RNAse_S', 'no_RNAse_G2','RNAse_G1','RNAse_S','RNAse_G2'] 
                         )

g2_terra_perc_df.set_index(['Cells'])#good way to fix "not in axis issue"
g2_terra_perc_df.dropna(axis=0, inplace=True) #sweet, iloc and drop didn't work, but this did


#g2_terra_perc_df.columns()
g2_terra_perc_df.head()

,Cells,no_RNAse_G1,no_RNAse_S,no_RNAse_G2,RNAse_G1,RNAse_S,RNAse_G2
0,EN-T,18.104772,19.965608,61.929620,24.736048,14.479638,60.784314
1,TRF1,17.160728,12.720786,70.118487,24.076535,8.397555,67.525910
2,Empty,22.765945,10.506481,66.727574,45.944346,11.870930,42.184725
3,Untransfected,39.699897,14.745256,45.554847,54.326047,16.029144,29.644809


# TERRA locolizes to Global DSBs
   - What the experiment is (e.g. what is it testing and how):
   - "To induce Global DSBs, U2OS cells were treated with 2 gy Gamma rays, recovered for 4 hours, fixed." 
   - "To evaluate the induction of TERRA in response to the Global DSBs, the fixed cells were stained with C-rich labeld PNA probe. The experiment was repeated and cells were treated with a cocktail of  Rnase H and A for the removal of TERRA foci (negative control)." 


# TERRA Locolizes to Telomeric DSBs
  - What the experiment is (e.g. what is it testing and how):
  - "To measure the colocalization of FLAG and TERRA  RNA FISH and Immunostaining to label FLAG in positively transfected cells and compare it to untransfected population."
  - to test wheather  this system allowed the formation of telomeric DSBs, cells were  fixed and immunostained to detect
    - a.FLAG  ( to label the positvely transfected cells).
    - b. yH2AX ( DSBs marker)
    - c. Edu ( S phase marker) FUCCI marker (G2 marker). 
    - The fixed cells were also stained with G-rich labeld PNA probe  to detect Telomeres (C-rich strands).


# TERRA locolization to Telomeric DSBs accumulated in the G2 phase of the cell cycle
  - Same experiment but assesed based on cells transfected with:
      - "G1 (stained  ONLY  with DAPI), in S phase (stained with DAPI+EdU) and G2 ( cells stained with green FUCCI and DAPI) and then compared this to the number of TERRA foci in untransfected control."
      - "To measure the colocalization of FLAG and TERRA  RNA FISH and Immunostaining to label FLAG in positively transfected cells and compare it to untransfected population."
      - to test wheather  this system allowed the formation of telomeric DSBs, cells were  fixed and immunostained to detect
        - a.FLAG  ( to label the positvely transfected cells).
        - b. yH2AX ( DSBs marker)
        - c. Edu ( S phase marker) FUCCI marker (G2 marker). 
        - The fixed cells were also stained with G-rich labeld PNA probe  to detect Telomeres (C-rich strands).

In [ ]:
g2_terra_num_df
g2_terra_perc_df
g2_cell_num_df
g2_cell_perc_df



# Conclusions 